<a href="https://colab.research.google.com/github/fatehOurghi/face-detection-evaluation/blob/main/notebook_FD_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation of face detection algorithms

This colab notebook will be used to evaluate different face detection algorithms including:
1.   TinaFace (best of Wider Face)
2.   DSFD (best of FDDB)
3.   SRN (best of Pascal Face)
---
Datasets used in this evaluation are:
*   Wider Face
*   FDDB
*   Pascal Face

**Machine configurations**

In [ ]:
!lsh

/bin/bash: lsh: command not found


# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Datasets Preparation

**Extract downloaded *Wider face* dataset**

In [ ]:
# Extract train data
import zipfile
dataset_path = "/content/drive/MyDrive/Datasets/FaceDetection/WiderFace/WIDER_train.zip"
unzip_to = "/content/drive/MyDrive/Datasets/WIDER_train"

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_to)

In [ ]:
# Extract test data
import zipfile
dataset_path = "/content/drive/MyDrive/Datasets/FaceDetection/WIDERFace/WIDER_test.zip"
unzip_to = "/content/drive/MyDrive/Datasets/FaceDetection/WIDERFace/test"

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_to)

In [ ]:
# Download dataset face annotations
!wget http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/bbx_annotation/wider_face_split.zip

In [ ]:
# extract annotations
!unzip -q /content/drive/MyDrive/Datasets/FaceDetection/WIDERFace/wider_face_split.zip

**Download of FDDB dataset**

In [ ]:
!wget http://vis-www.cs.umass.edu/fddb/originalPics.tar.gz

In [ ]:
!wget http://vis-www.cs.umass.edu/fddb/FDDB-folds.tgz

**Extract FDDB dataset**




In [ ]:
# extract dataset
!tar -zxvf /content/drive/MyDrive/Datasets/FaceDetection/PascalFace/caltech.tar.gz

In [ ]:
# extract annotations
!tar -zxvf /content/drive/MyDrive/Datasets/FaceDetection/FDDB/FDDB-folds.tgz

**Extract FDDB dataset**

In [ ]:
!tar -zxvf /content/drive/MyDrive/Datasets/FaceDetection/FDDB/originalPics.tar.gz

**Download the Pascal Face dataset**

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/download/caltech.tar.gz

# Initialize global variables

In [ ]:
DATASET = "WIDERFace"
##########################################################################
DATASET_PATH = f"/content/drive/MyDrive/Datasets/FaceDetection/{DATASET}/"
train_path = f"{DATASET_PATH}train/"
test_path = f"{DATASET_PATH}test/"

# Load dataset

**Load WiderFace**

**Load FDDB**

**Load Pascal Face**

# Prepare dataset

# Test Algorithms

annotations  test  train
